In [23]:
import pandas as pd

In [24]:
df = pd.read_csv('./data/survey_results_public.csv')
df_schema = pd.read_csv('./data/survey_results_schema.csv')

In [25]:
# filtering according to country and purchase influence columns

high_influence_df = df[(df['Country'] == 'Iran, Islamic Republic of...') & (df['PurchaseInfluence'] == 'I have a great deal of influence')]
high_influence_df

In [27]:
# filtering according to country and purchase influence columns

some_influence_df = df[(df['Country'] == 'Iran, Islamic Republic of...') & (df['PurchaseInfluence'] == 'I have some influence')]
some_influence_df

In [29]:
# table of employment distribution for developers with high influence

high_influence_employment = high_influence_df['Employment'].value_counts()
high_influence_employment

Employment
Employed, full-time                                                                             70
Independent contractor, freelancer, or self-employed                                            27
Employed, full-time;Independent contractor, freelancer, or self-employed                         7
Independent contractor, freelancer, or self-employed;Employed, part-time                         5
Employed, full-time;Employed, part-time                                                          2
Employed, part-time                                                                              2
I prefer not to say                                                                              2
Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time     1
Name: count, dtype: int64

In [30]:
# table of employment distribution for developers with some influence

some_influence_employment = some_influence_df['Employment'].value_counts()
some_influence_employment

Employment
Employed, full-time                                                                             132
Independent contractor, freelancer, or self-employed                                             16
Employed, full-time;Independent contractor, freelancer, or self-employed                         13
Employed, part-time                                                                               5
Employed, full-time;Independent contractor, freelancer, or self-employed;Employed, part-time      4
I prefer not to say                                                                               1
Employed, full-time;Employed, part-time                                                           1
Name: count, dtype: int64